In [1]:
!pip install gurobipy

In [2]:
from gurobipy import *

## 問題例5 手数に上限を加えて制約を満たしていないものにペナルティを加える。

In [49]:
def order(a,b):
  return [a,b]

In [50]:
# nと初期配置を変更しなければいけない
O=order(1,2)
S=[i for i in range(1,3)]
H=[i for i in range(1,3)]
P=[i for i in range(1,3)]
T=[i for i in range(1,7)]
n1=[1,2]
n=[n1[O[0]-1],n1[O[1]-1]]

S,H,P,T,n

([1, 2], [1, 2], [1, 2], [1, 2, 3, 4, 5, 6], [1, 2])

In [63]:
m=Model("IP3")

In [64]:
x,y,pre,penalty={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")

for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


# for s in S:
#   for h in range(1,len(H)):
#     for p in P:
#       pre[s,h,p]=m.addVar(vtype="C")
#       penalty[s,h,p]=m.addVar(vtype="c")



In [65]:
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

In [66]:
for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])

In [67]:
for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))


# for s in S:
#   for h in range(1,len(H)):
#     for p in P:
#       L= [0,quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))-quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1))]
#       m.addConstr(penalty[s,h,p]==max(L))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

In [68]:
# m.addConstr(x[1,1,O[3],1]==1)
# m.addConstr(x[1,2,O[3],1]==1)
# m.addConstr(x[1,3,O[2],1]==1)
# m.addConstr(x[1,4,O[1],1]==1)
# m.addConstr(x[2,1,O[3],1]==1)
# m.addConstr(x[2,2,O[2],1]==1)
# m.addConstr(x[2,3,O[1],1]==1)
# m.addConstr(x[2,4,O[0],1]==1)
# m.addConstr(x[3,1,O[2],1]==1)
# m.addConstr(x[3,2,O[2],1]==1)

m.addConstr(x[1,1,O[0],1]==1)
m.addConstr(x[1,2,O[1],1]==1)
m.addConstr(x[2,1,O[1],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [70]:
for s in S:
  for h in range(1,len(H)):
    for p in P:
      # m.addConstr(pre[s,h,p]==quicksum(x[s,h+1,k,len(T)-1] for k in range(p,len(P)+1))-quicksum(x[s,h,k,len(T)-1] for k in range(p,len(P)+1)))
      a=[0]
      pre[s,h,p]=sum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))-sum(x[s,h,k,len(T)] for k in range(p,len(P)+1))
      a.append(pre[s,h,p])
      penalty[s,h,p]=max_(a)
      # print(penalty[s,h,p])


In [71]:
m.setObjective(quicksum(y[s,k,t] for s in S for k in S if k!=s for t in range(1,len(T)))
+1.1*sum(penalty[s,h,p] for s in S for h in H if h!=len(H) for p in P))
m.optimize()
print(m.Status)

TypeError: unsupported operand type(s) for +: 'int' and 'GenExprMax'

In [32]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    # if t==1:
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    # if t>=2:
    #   if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t==len(T):
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))
  print("the number of moves", m.objVal)